# Holistic Path Benchmark
In this experiment we will try to produce comparable results between various learning methods. We will use Holistic Path as our task and train a centralised, federated, swarm, and federated baseline model. We will then compare the results of these models to see how decentralised learning compares to centralised learning for this specific task.

Our configuration for this specific setup is as follows:
- Balanced Dataset
- 40% of the balanced dataset is used
- ROI frame cropping
- Pre-trained ResNet18
- 5 Clients
- Swarm setup is a fully connected static network

Below is the full configuration


In [1]:
import json

config = {
    "data":{
        "path": "/mnt/ZOD",
        "version":"full",
        "ratio":0.5,
        "img_size": [400, 1420],
        "transforms":"[CustomCrop(800, 500, 800, 2840), Resize((400, 1420))]",
        "shuffle_seed": 42,
        "decentralised_shuffle_seed": 42,
        "dataloader_args": {
            "prefetch_factor": 3,
            "num_workers": 8,
            "batch_size": 64,
        },
        "train_val_id_generator":"balanced_frames",
        "dataset_getitem":"interpolated_target_distances",
    },
    "model":{
        "name": "ptnet",
        "args": {
            "num_output":66
        },
        "loss":"MSELoss"
    },
    "central":{
        "train":"true",
        "use_gpu":"true",
        "epochs": 15
    },
    "decentralised":{
        "train": [],
        "global":{
            "n_clients":5,
            "global_rounds":3,
            "client_resources":{
                "num_cpus": 2, 
                "num_gpus": 0.3
            },
            "ray_init_args":{
                "include_dashboard": True,
                "num_cpus": 2,
                "num_gpus": 2
            },
            "swarm_orchestrator": "synchronous_fixed_rounds_fc",
        },
        "client": {
            "epochs": 5
        }
    },
    "note": "Balanced frames, ROI, 50 epochs, 5 clients with pretrained model"
}


with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

In [ ]:
!nohup python ../../platform/run_platform.py --config=config.json &

# Results
Report here

In [1]:
import json

# test
config = {
    "data":{
        "path": "/mnt/ZOD",
        "version":"full",
        "ratio":0.4,
        "img_size": [400, 1420],
        "transforms":"[CustomCrop(800, 500, 800, 2840), Resize((400, 1420))]",
        "shuffle_seed": 42,
        "decentralised_shuffle_seed": 42,
        "dataloader_args": {
            "batch_size": 32,
        },
        "train_val_id_generator":"balanced_frames"
    },
    "model":{
        "name": "ptnet",
        "args": {
            "num_output":66
        },
        "loss":"MSELoss"
    },
    "central":{
        "train":"false",
        "use_gpu":"true",
        "epochs": 15
    },
    "decentralised":{
        "train": ["swarm"],
        "global":{
            "n_clients": 5,
            "global_rounds": 3,
            "client_resources":{
                "num_cpus": 1, 
                "num_gpus": 0.3
            },
            "ray_init_args":{
                "include_dashboard": True,
                "num_cpus": 6,
                "num_gpus": 2
            }
        },
        "client": {
            "epochs": 3
        }
    },
    "note": "Balanced frames, ROI, 50 epochs, 5 clients with pretrained model"
}


with open("config.json", "w") as f:
    json.dump(config, f, indent=4)